# Analysing the Datasets

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp

from tensorflow import keras


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
boston_dataset = keras.datasets.boston_housing

In [3]:
(train_data,train_label), (test_data, test_label) = boston_dataset.load_data()

In [4]:
train_label = train_label[np.newaxis].T

In [5]:
total_train_data = train_data
total_train_data = np.append(total_train_data,train_label, axis=1)

In [6]:
correlation_matrix = tfp.stats.correlation(total_train_data, sample_axis=0, event_axis=-1)

Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().


In [7]:
X = tf.placeholder(tf.float64, [None, 2])
Y = tf.placeholder(tf.float64, [None, 1])

In [8]:
W = tf.Variable(tf.zeros([2, 1], dtype=tf.float64))
B = tf.Variable(tf.zeros([1], dtype=tf.float64))

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
Y1 = tf.add(tf.matmul(X,W),B)

In [10]:
cost = tf.reduce_mean(tf.square(Y-Y1))

In [11]:
train_step = tf.train.GradientDescentOptimizer(0.000001).minimize(cost) 

In [12]:
init = tf.global_variables_initializer()

In [13]:
sess = tf.Session()

In [14]:
sess.run(init)

In [15]:
corr_matrix = sess.run(correlation_matrix)

In [16]:
#This method takes a correlation array as an input and outputs a heatmap
def correlation_matrix(df):
    from matplotlib import pyplot as plt
    from matplotlib import cm as cm

    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    cmap = cm.get_cmap('jet', 30)
    cax = ax1.imshow(df, interpolation="nearest", cmap=cmap)
    ax1.grid(True)
    # Add colorbar, make sure to specify tick locations to match desired ticklabels
    fig.colorbar(cax, ticks=[-1, -0.75, -0.5, -0.25, 0,.25,.50,.75,1])
    plt.show()
correlation_matrix(corr_matrix)

<Figure size 640x480 with 2 Axes>

In [17]:
#This method derived the parameters post analyzing the correlation metrics
def getCorrelationParameters(correlation_matrix,label_index = -1 , correlation_threshold = 0.5):
    if label_index == -1:
        label_index = len(correlation_matrix) - 1
    targetParams = [i for i,x in enumerate(correlation_matrix[label_index]) if (abs(x) >= abs(correlation_threshold)) and (i != label_index)]
    exclude = []
    for i in range(0, len(targetParams)):
        for j in range(i, len(targetParams)):
            if (i != j) and (abs(correlation_matrix[i][j]) >= abs(correlation_threshold)):
                param = i if correlation_matrix[label_index][i] >= correlation_matrix[label_index][j] else j
                if (i not in exclude) and (j not in exclude):
                    exclude.append(param)
    
    result =  list(set(targetParams) - set(exclude))
    result.sort()
    return result
meaningful_fields = getCorrelationParameters(corr_matrix)

In [18]:
def reduceDataframe(array, columns):
    arrayReduced = array[:, columns]
    return arrayReduced
train_data = reduceDataframe(total_train_data, meaningful_fields)

In [19]:
train_data.shape

(404, 2)

In [20]:
feed = {X: train_data, Y: train_label}

In [21]:
epoch = 10000000
for i in range(0,epoch):
    #print("Iteration: {}".format(i))
    sess.run(train_step,feed_dict = feed)
    #print("W: {}".format(sess.run(W)))
    #print("B: {}".format(sess.run(B)))
print("W: {}".format(sess.run(W)))
print("B: {}".format(sess.run(B)))

W: [[ 4.78731617]
 [-0.64254939]]
B: [0.58262141]


In [22]:
w = sess.run(W)
b = sess.run(B)

In [23]:
test_data = reduceDataframe(test_data, meaningful_fields)

In [24]:
error = 0
for i in range(test_data.shape[0]):
    #print(sess.run(tf.add(tf.matmul(w.T, test_data[i][np.newaxis].T), b))[0][0], test_label[i])
    error = pow(sess.run(tf.add(tf.matmul(w.T, test_data[i][np.newaxis].T), b))[0][0] - test_label[i], 2)
error /= test_data.shape[0]
print(pow(error,0.5))

0.1498461710962739
